# 1. Extract Boundary Tone Feature from Korean Speech Dataset
- Intonation Annotation through pitch slope in *Intonation Phrase (IP)*

- 파일 구조:

  input_audio_dir: 원본 .wav 파일이 저장된 디렉터리.

  input_json_dir: JSON 라벨 파일이 저장된 디렉터리.
  
  output_audio_dir: 분리된 발화 .wav 파일을 저장할 디렉터리.
  
  output_csv_path: 최종 결과를 저장할 CSV 파일 경로.
- JSON에서 발화 정보 추출:

  발화의 시작 시간 (StartTime)과 종료 시간 (EndTime)을 기준으로 원본 오디오 파일에서 발화를 분리.
- Pitch 기울기와 억양 주석 계산:

  librosa를 사용해 pitch 데이터를 추출하고, 기울기 및 억양 주석 정보를 생성.
- 결과 저장:

  분리된 발화 오디오 파일 이름, 감정 레이블 (emotion), pitch 기울기, 억양 주석 정보를 포함한 결과를 DataFrame으로 정리하고 CSV로 저장.
- 결과 CSV 파일 컬럼:

  ID: 발화별로 생성된 새로운 .wav 파일 이름.
  
  emotion: JSON에서 추출한 감정 레이블.
  
  pitch_slope: 발화별 pitch 기울기 데이터 (리스트).
  
  intonation_annotation: 발화별 억양 주석 데이터 (리스트).

In [ ]:
!pip install praat-parselmouth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 18.7 MB/s eta 0:00:00


In [ ]:
import os
import zipfile

import json
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
import parselmouth
import matplotlib.pyplot as plt

In [ ]:
# 압축 파일 경로
zip_file_audio = "/content/train.zip"  # 첫 번째 압축 파일 경로
zip_file_labeling = "/content/train_labeling.zip"  # 두 번째 압축 파일 경로

In [ ]:
# 파일이 올바른 zip 파일인지 확인하는 함수
def check_zip_file(file_path):
    try:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # zip 파일 내용 확인
            zip_ref.testzip()
        return True
    except zipfile.BadZipFile:
        print(f"Error: {file_path} is not a valid zip file.")
        return False

# 압축 해제할 경로 설정
extract_audio_dir = "/content/extracted_audio_files"  # 오디오 파일 압축 해제 경로
extract_labeling_dir = "/content/extracted_labeling_files"  # 라벨링 파일 압축 해제 경로

# 파일 경로 체크 후 압축 해제
if check_zip_file(zip_file_audio):
    os.makedirs(extract_audio_dir, exist_ok=True)
    with zipfile.ZipFile(zip_file_audio, 'r') as zip_ref:
        zip_ref.extractall(extract_audio_dir)  # 오디오 파일 압축 해제

if check_zip_file(zip_file_labeling):
    os.makedirs(extract_labeling_dir, exist_ok=True)
    with zipfile.ZipFile(zip_file_labeling, 'r') as zip_ref:
        zip_ref.extractall(extract_labeling_dir)  # 라벨링 파일 압축 해제

# 경로 설정
input_audio_dir = extract_audio_dir  # 원본 .wav 파일이 포함된 폴더 경로
input_json_dir = extract_labeling_dir  # JSON 파일이 포함된 폴더 경로
output_audio_dir = "/content/output_wavs"  # 발화별 .wav 파일 저장 경로
os.makedirs(output_audio_dir, exist_ok=True)

output_csv_path = "/content/utterance_features.csv"  # 최종 .csv 파일 경로

# 경로 확인
print("원본 오디오 파일 경로:", input_audio_dir)
print("라벨링 JSON 파일 경로:", input_json_dir)
print("발화별 오디오 저장 경로:", output_audio_dir)
print("최종 CSV 파일 경로:", output_csv_path)


원본 오디오 파일 경로: /content/extracted_audio_files
라벨링 JSON 파일 경로: /content/extracted_labeling_files
발화별 오디오 저장 경로: /content/output_wavs
최종 CSV 파일 경로: /content/utterance_features.csv


In [ ]:
import json
import os
import librosa
import soundfile as sf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.interpolate import interp1d
import parselmouth
import matplotlib.pyplot as plt

# 피치 기울기 및 억양 주석 함수
def calculate_pitch_slope(pitch_times, pitch_values, start_time, end_time):
    within_range = (pitch_times >= start_time) & (pitch_times <= end_time)
    segment_times = pitch_times[within_range]
    segment_values = pitch_values[within_range]

    if len(segment_times) < 2 or np.isnan(segment_values).all():
        return None, None, "Insufficient data"

    model = LinearRegression()
    model.fit(segment_times.reshape(-1, 1), segment_values)
    slope = model.coef_[0]
    return slope, segment_values, model, "Slope calculated"

def annotate_intonation(slope, segment_values, full_pitch_values, rise_threshold=0.5, flat_threshold=0.1):
    if slope is None:
        return "No annotation (insufficient data)"
    if slope > rise_threshold:
        return "H%"
    elif slope < -rise_threshold:
        return "L%"
    elif abs(slope) <= flat_threshold:
        full_mean_pitch = np.nanmean(full_pitch_values)
        segment_mean_pitch = np.nanmean(segment_values)
        return "H%" if segment_mean_pitch > full_mean_pitch else "L%"
    else:
        return "Flat"

def process_and_annotate_pitch(
    sound_path,
    last_phrase_start=1.943,  # Hardcoded start time for last phrase
    time_step=0.01,
    pitch_floor=100,
    pitch_ceiling=450,
    rise_threshold=0.5,
    flat_threshold=0.1
):
    """
    Process the sound file, calculate pitch slope, and annotate intonation.
    """
    sound = parselmouth.Sound(sound_path)
    pitch = sound.to_pitch(time_step=time_step, pitch_floor=pitch_floor, pitch_ceiling=pitch_ceiling)
    pitch_values = pitch.selected_array['frequency']
    pitch_times = pitch.xs()
    pitch_values[pitch_values == 0] = np.nan

    # 유효한 값만 필터링
    valid_idx = ~np.isnan(pitch_values)

    # 유효한 값이 하나도 없는 경우 처리
    if valid_idx.sum() == 0:
        print(f"No valid pitch data for {sound_path} between {last_phrase_start} and {pitch_times[-1]}.")
        return None, "No valid pitch data"

    # 보간 함수 정의
    interp_function = interp1d(
        pitch_times[valid_idx],
        pitch_values[valid_idx],
        kind="linear",
        bounds_error=False,
        fill_value=np.nan
    )

    # 보간된 피치 값 생성
    interpolated_pitch = interp_function(pitch_times)

    # 기울기 계산
    slope, segment_values, model, status = calculate_pitch_slope(pitch_times, interpolated_pitch, last_phrase_start, pitch_times[-1])

    if slope is None:
        annotation = "No annotation (insufficient data)"
    else:
        annotation = annotate_intonation(slope, segment_values, interpolated_pitch)

    return slope, annotation





In [ ]:
# 발화 단위로 .wav 파일 분리 및 정보 추출
results = []

for json_file in os.listdir(input_json_dir):
    if not json_file.endswith(".json"):
        continue

    with open(os.path.join(input_json_dir, json_file), "r", encoding="utf-8") as f:
        data = json.load(f)

    audio_file_name = data["File"]["FileName"] + ".wav"
    audio_file_path = os.path.join(output_audio_dir, audio_file_name)

    if not os.path.exists(audio_file_path):
        print(f"Audio file not found: {audio_file_path}")
        continue

    y, sr = librosa.load(audio_file_path, sr=None)

    for utterance in data["Conversation"]:
        try:
            start_time = float(utterance["StartTime"].replace(",", ""))
            end_time = float(utterance["EndTime"].replace(",", ""))
        except ValueError as e:
            print(f"Error converting times for {utterance['TextNo']}: {e}")
            continue  # 시간 변환에 실패하면 해당 발화를 건너뜁니다.

        emotion = utterance["SpeakerEmotionTarget"]
        utterance_id = utterance["TextNo"]

        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        y_segment = y[start_sample:end_sample]

        segment_file_name = f"{data['File']['FileName']}_{utterance_id}.wav"
        segment_file_path = os.path.join(output_audio_dir, segment_file_name)
        sf.write(segment_file_path, y_segment, sr)

        slope, annotation = process_and_annotate_pitch(segment_file_path, last_phrase_start=start_time)
        results.append({
            "ID": segment_file_name,
            "emotion": emotion,
            "pitch_slope": slope,
            "intonation_annotation": annotation
        })


# 결과를 DataFrame으로 저장 및 CSV로 내보내기
df = pd.DataFrame(results)
df.to_csv(output_csv_path, index=False)

print(f"처리가 완료되었습니다. 결과는 {output_csv_path}에 저장되었습니다.")

### 마지막 음절 구간 추출 후 억양 주석 달도록 수정

In [ ]:
import os
import json
import numpy as np
import parselmouth
from sklearn.linear_model import LinearRegression
from scipy.interpolate import interp1d

def get_last_syllable_times(sound_path, time_step=0.01, pitch_floor=75, pitch_ceiling=500):
    """
    주어진 음성 파일에서 마지막 음절의 시작 시간과 끝 시간을 추출.
    """
    # Praat의 Sound 객체 생성
    sound = parselmouth.Sound(sound_path)

    # 피치 분석
    pitch = sound.to_pitch(time_step=time_step, pitch_floor=pitch_floor, pitch_ceiling=pitch_ceiling)
    pitch_values = pitch.selected_array['frequency']
    pitch_times = pitch.xs()

    # 유효한 피치 값만 필터링
    valid_idx = ~np.isnan(pitch_values)
    valid_times = pitch_times[valid_idx]

    if len(valid_times) == 0:
        print(f"유효한 피치 값이 없습니다: {sound_path}")
        return None, None

    # 마지막 피치 구간 찾기
    last_pitch_time = valid_times[-1]

    # 마지막 음절의 시작 시간 추정
    start_time = last_pitch_time - (time_step * 2)  # 대략적인 범위 설정
    start_time = max(0, start_time)  # 0 이하 방지

    return start_time, last_pitch_time

def calculate_pitch_slope(pitch_times, pitch_values, start_time, end_time):
    """
    피치 구간의 기울기를 계산.
    """
    within_range = (pitch_times >= start_time) & (pitch_times <= end_time)
    segment_times = pitch_times[within_range]
    segment_values = pitch_values[within_range]

    if len(segment_times) < 2 or np.isnan(segment_values).all():
        return None, None, "Insufficient data"

    model = LinearRegression()
    model.fit(segment_times.reshape(-1, 1), segment_values)
    slope = model.coef_[0]
    return slope, segment_values, model, "Slope calculated"

def annotate_intonation(slope, segment_values, full_pitch_values, rise_threshold=0.5, flat_threshold=0.1):
    """
    피치 기울기를 기반으로 억양 주석을 생성.
    """
    if slope is None:
        return "No annotation (insufficient data)"
    if slope > rise_threshold:
        return "H%"
    elif slope < -rise_threshold:
        return "L%"
    elif abs(slope) <= flat_threshold:
        full_mean_pitch = np.nanmean(full_pitch_values)
        segment_mean_pitch = np.nanmean(segment_values)
        return "H%" if segment_mean_pitch > full_mean_pitch else "L%"
    else:
        return "Flat"

def process_and_annotate_pitch(
    sound_path,
    last_phrase_start,
    time_step=0.01,
    pitch_floor=100,
    pitch_ceiling=450,
    rise_threshold=0.5,
    flat_threshold=0.1
):
    """
    억양 주석을 생성하는 함수.
    """
    sound = parselmouth.Sound(sound_path)
    pitch = sound.to_pitch(time_step=time_step, pitch_floor=pitch_floor, pitch_ceiling=pitch_ceiling)
    pitch_values = pitch.selected_array['frequency']
    pitch_times = pitch.xs()
    pitch_values[pitch_values == 0] = np.nan

    valid_idx = ~np.isnan(pitch_values)

    if valid_idx.sum() == 0:
        print(f"No valid pitch data for {sound_path}.")
        return None, "No valid pitch data"

    interp_function = interp1d(
        pitch_times[valid_idx],
        pitch_values[valid_idx],
        kind="linear",
        bounds_error=False,
        fill_value=np.nan
    )

    interpolated_pitch = interp_function(pitch_times)

    end_time = pitch_times[-1]  # 마지막 피치 시간
    slope, segment_values, status = calculate_pitch_slope(pitch_times, interpolated_pitch, last_phrase_start, end_time)

    if slope is None:
        annotation = "No annotation (insufficient data)"
    else:
        annotation = annotate_intonation(slope, segment_values, interpolated_pitch, rise_threshold, flat_threshold)

    return slope, annotation

# 전체 프로세스 수행
input_audio_dir = "/path/to/audio_files"
output_json_path = "/path/to/output.json"

results = []
for audio_file in os.listdir(input_audio_dir):
    if not audio_file.endswith(".wav"):
        continue

    audio_path = os.path.join(input_audio_dir, audio_file)

    # 마지막 음절 구간 추출
    start_time, end_time = get_last_syllable_times(audio_path)

    if start_time is None or end_time is None:
        print(f"Skipping {audio_file} due to insufficient data.")
        continue

    # 억양 주석 처리
    slope, annotation = process_and_annotate_pitch(audio_path, last_phrase_start=start_time)
    results.append({
        "audio_file": audio_file,
        "last_syllable_start_time": start_time,
        "last_syllable_end_time": end_time,
        "pitch_slope": slope,
        "intonation_annotation": annotation
    })

# 결과 저장
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"처리가 완료되었습니다. 결과는 {output_json_path}에 저장되었습니다.")


In [ ]:
# 발화 단위로 .wav 파일 분리 및 정보 추출
results = []

for json_file in os.listdir(input_json_dir):
    if not json_file.endswith(".json"):
        continue

    with open(os.path.join(input_json_dir, json_file), "r", encoding="utf-8") as f:
        data = json.load(f)

    audio_file_name = data["File"]["FileName"] + ".wav"
    audio_file_path = os.path.join(output_audio_dir, audio_file_name)

    if not os.path.exists(audio_file_path):
        print(f"Audio file not found: {audio_file_path}")
        continue

    y, sr = librosa.load(audio_file_path, sr=None)

    for utterance in data["Conversation"]:
        try:
            start_time = float(utterance["StartTime"].replace(",", ""))
            end_time = float(utterance["EndTime"].replace(",", ""))
        except ValueError as e:
            print(f"Error converting times for {utterance['TextNo']}: {e}")
            continue  # 시간 변환에 실패하면 해당 발화를 건너뜁니다.

        emotion = utterance["SpeakerEmotionTarget"]
        utterance_id = utterance["TextNo"]

        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        y_segment = y[start_sample:end_sample]

        segment_file_name = f"{data['File']['FileName']}_{utterance_id}.wav"
        segment_file_path = os.path.join(output_audio_dir, segment_file_name)
        sf.write(segment_file_path, y_segment, sr)

        slope, annotation = process_and_annotate_pitch(segment_file_path, last_phrase_start=start_time)
        results.append({
            "ID": segment_file_name,
            "emotion": emotion,
            "pitch_slope": slope,
            "intonation_annotation": annotation
        })


# 결과를 DataFrame으로 저장 및 CSV로 내보내기
df = pd.DataFrame(results)
df.to_csv(output_csv_path, index=False)

print(f"처리가 완료되었습니다. 결과는 {output_csv_path}에 저장되었습니다.")

### 수정

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import parselmouth
import librosa
import soundfile as sf
from sklearn.linear_model import LinearRegression
from scipy.interpolate import interp1d

# praat -> librosa 변경
import librosa
import numpy as np

### 에러

- 마지막 음절 구간을 정확히 추출할 수 없어, 피치 데이터를 사용하려고 하면 에러가 남
- 우선 시간이 아닌, 유효한 피치 중 마지막 추출 범위를 인식하도록 하겠음

In [ ]:


# def get_last_syllable_times(sound_path, time_step=0.01, pitch_floor=75, pitch_ceiling=500):
#     """
#     주어진 음성 파일에서 마지막 음절의 시작 시간과 끝 시간을 추출.
#     """
#     sound = parselmouth.Sound(sound_path)
#     pitch = sound.to_pitch(time_step=time_step, pitch_floor=pitch_floor, pitch_ceiling=pitch_ceiling)
#     pitch_values = pitch.selected_array['frequency']
#     pitch_times = pitch.xs()

#     valid_idx = ~np.isnan(pitch_values)
#     valid_times = pitch_times[valid_idx]

#     if len(valid_times) == 0:
#         print(f"유효한 피치 값이 없습니다: {sound_path}")
#         return None, None

#     last_pitch_time = valid_times[-1]
#     start_time = max(0, last_pitch_time - (time_step * 2))  # 대략적인 범위 설정

#     return start_time, last_pitch_time

import librosa
import numpy as np
import soundfile as sf

def get_last_syllable_times(sound_path, time_step=0.01, pitch_floor=75, pitch_ceiling=500):
    try:
        # 음성 파일 로드
        y, sr = librosa.load(sound_path, sr=None)

        # onset_env는 음성에서 발음이 시작되는 지점 탐지
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)

        # 피치 계산: piptrack은 각 시간 단계에서 피치를 추출
        pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)

        # 각 타임스탬프에서 가장 강한 피치를 추출
        pitch_values = [pitches[i, mag.argmax()] for i, mag in enumerate(magnitudes.T) if mag.any()]

        if not pitch_values:
            print(f"No valid pitch data for {sound_path}")
            return None, None

        # 피치가 존재하는 데이터의 시간 범위를 계산
        pitch_times = librosa.frames_to_time(range(len(pitch_values)), sr=sr, hop_length=1024)

        # 피치 값이 0인 곳을 NaN으로 처리
        pitch_values = np.array(pitch_values)
        pitch_values[pitch_values == 0] = np.nan

        # 유효한 피치 값의 인덱스
        valid_idx = ~np.isnan(pitch_values)

        # 유효한 피치 값이 없으면 처리하지 않음
        if valid_idx.sum() == 0:
            print(f"Not enough valid pitch data for {sound_path}")
            return None, None

        # 유효한 피치 범위 계산
        valid_pitch_times = pitch_times[valid_idx]
        valid_pitch_values = pitch_values[valid_idx]

        # 마지막 유효한 피치 데이터의 시간 범위
        last_syllable_end = valid_pitch_times[-1]  # 마지막 유효 피치 시간
        last_syllable_start = valid_pitch_times[0]  # 첫 번째 유효 피치 시간

        # 마지막 음절의 끝을 유효한 피치 범위로 설정
        if last_syllable_end > valid_pitch_times[-1]:
            last_syllable_end = valid_pitch_times[-1]

        return last_syllable_start, last_syllable_end

    except Exception as e:
        print(f"Error processing {sound_path}: {e}")
        return None, None


def calculate_pitch_slope(pitch_times, pitch_values, start_time, end_time):
    """
    피치 구간의 기울기를 계산.
    """
    within_range = (pitch_times >= start_time) & (pitch_times <= end_time)
    segment_times = pitch_times[within_range]
    segment_values = pitch_values[within_range]

    # 에러: NaN 값 제거 코드 추가
    valid_indices = ~np.isnan(segment_values)
    segment_times = segment_times[valid_indices]
    segment_values = segment_values[valid_indices]

    if len(segment_times) < 2 or len(segment_values) < 2:
        return None, None, "Insufficient data"  # 유효한 데이터가 부족한 경우 처리

    # if len(segment_times) < 2 or np.isnan(segment_values).all():
    #     return None, None, "Insufficient data"

    model = LinearRegression()
    model.fit(segment_times.reshape(-1, 1), segment_values)
    slope = model.coef_[0]
    return slope, segment_values, "Slope calculated" # 일단 모델 뺌



def annotate_intonation(slope, segment_values, full_pitch_values, rise_threshold=0.5, flat_threshold=0.1):
    """
    피치 기울기를 기반으로 억양 주석을 생성.
    """
    if slope is None:
        return "No annotation (insufficient data)"
    if slope > rise_threshold:
        return "H%"
    elif slope < -rise_threshold:
        return "L%"
    elif abs(slope) <= flat_threshold:
        full_mean_pitch = np.nanmean(full_pitch_values)
        segment_mean_pitch = np.nanmean(segment_values)
        return "H%" if segment_mean_pitch > full_mean_pitch else "L%"
    else:
        return "Flat"

def process_and_annotate_pitch(
    sound_path,
    last_phrase_start,
    time_step=0.01,
    pitch_floor=100,
    pitch_ceiling=450,
    rise_threshold=0.5,
    flat_threshold=0.1
):
    """
    억양 주석을 생성하는 함수.
    """
    sound = parselmouth.Sound(sound_path)
    pitch = sound.to_pitch(time_step=time_step, pitch_floor=pitch_floor, pitch_ceiling=pitch_ceiling)
    pitch_values = pitch.selected_array['frequency']
    pitch_times = pitch.xs()
    pitch_values[pitch_values == 0] = np.nan

    valid_idx = ~np.isnan(pitch_values)

    if valid_idx.sum() == 0:
        print(f"No valid pitch data for {sound_path}.")
        return None, "No valid pitch data"

    interp_function = interp1d(
        pitch_times[valid_idx],
        pitch_values[valid_idx],
        kind="linear",
        bounds_error=False,
        fill_value=np.nan
    )

    interpolated_pitch = interp_function(pitch_times)

    end_time = pitch_times[-1]  # 마지막 피치 시간
    slope, segment_values, status = calculate_pitch_slope(pitch_times, interpolated_pitch, last_phrase_start, end_time)
    # 모델 제외하고 받음

    if slope is None : # or segment_values is None: 원래대로
        annotation = "No annotation (insufficient data)"
    else:
        annotation = annotate_intonation(slope, segment_values, interpolated_pitch, rise_threshold, flat_threshold)

    return slope, annotation




In [ ]:
# 경로 설정
input_audio_dir = "/content/extracted_audio_files"
input_json_dir = "/content/extracted_labeling_files"
output_audio_dir = "/content/output_wavs"
os.makedirs(output_audio_dir, exist_ok=True)

output_csv_path = "/content/utterance_features.csv"

In [ ]:
# 발화 단위로 .wav 파일 분리 및 정보 추출
results = []

for json_file in os.listdir(input_json_dir):
    if not json_file.endswith(".json"):
        continue

    with open(os.path.join(input_json_dir, json_file), "r", encoding="utf-8") as f:
        data = json.load(f)

    audio_file_name = data["File"]["FileName"] + ".wav"
    audio_file_path = os.path.join(input_audio_dir, audio_file_name)

    if not os.path.exists(audio_file_path):
        print(f"Audio file not found: {audio_file_path}")
        continue

    y, sr = librosa.load(audio_file_path, sr=None)

    for utterance in data["Conversation"]:
        try:
            start_time = float(utterance["StartTime"].replace(",", ""))
            end_time = float(utterance["EndTime"].replace(",", ""))
        except ValueError as e:
            print(f"Error converting times for {utterance['TextNo']}: {e}")
            continue

        emotion = utterance["SpeakerEmotionTarget"]
        utterance_id = utterance["TextNo"]

        start_sample = int(start_time * sr)
        end_sample = int(end_time * sr)
        y_segment = y[start_sample:end_sample]

        segment_file_name = f"{data['File']['FileName']}_{utterance_id}.wav"
        segment_file_path = os.path.join(output_audio_dir, segment_file_name)
        sf.write(segment_file_path, y_segment, sr)

        last_syllable_start, last_syllable_end = get_last_syllable_times(segment_file_path)
        if last_syllable_start is not None and last_syllable_end is not None:
            print(f"{segment_file_path}: Last syllable time range is from {last_syllable_start:.2f}s to {last_syllable_end:.2f}s")
        else:
            print(f"{segment_file_path}: Skipped due to insufficient pitch data.")



        if last_syllable_start is None or last_syllable_end is None:
            print(f"Skipping {segment_file_name} due to insufficient data.")
            continue

        slope, annotation = process_and_annotate_pitch(segment_file_path, last_phrase_start=last_syllable_start)
        results.append({
            "ID": segment_file_name,
            "emotion": emotion,
            "last_syllable_start_time": last_syllable_start,
            "last_syllable_end_time": last_syllable_end,
            "pitch_slope": slope,
            "intonation_annotation": annotation
        })

# 결과를 DataFrame으로 저장 및 CSV로 내보내기
df = pd.DataFrame(results)
df.to_csv(output_csv_path, index=False)

print(f"처리가 완료되었습니다. 결과는 {output_csv_path}에 저장되었습니다.")

Error processing /content/output_wavs/2_1397G1A4_1398G2A6_T1_2D02T0062C000696_005160_000001.wav: index 39 is out of bounds for axis 1 with size 39
/content/output_wavs/2_1397G1A4_1398G2A6_T1_2D02T0062C000696_005160_000001.wav: Skipped due to insufficient pitch data.
Skipping 2_1397G1A4_1398G2A6_T1_2D02T0062C000696_005160_000001.wav due to insufficient data.
Error processing /content/output_wavs/2_1397G1A4_1398G2A6_T1_2D02T0062C000696_005160_000002.wav: index 69 is out of bounds for axis 1 with size 42
/content/output_wavs/2_1397G1A4_1398G2A6_T1_2D02T0062C000696_005160_000002.wav: Skipped due to insufficient pitch data.
Skipping 2_1397G1A4_1398G2A6_T1_2D02T0062C000696_005160_000002.wav due to insufficient data.
Error processing /content/output_wavs/2_1397G1A4_1398G2A6_T1_2D02T0062C000696_005160_000003.wav: index 481 is out of bounds for axis 1 with size 41
/content/output_wavs/2_1397G1A4_1398G2A6_T1_2D02T0062C000696_005160_000003.wav: Skipped due to insufficient pitch data.
Skipping 2_1

KeyboardInterrupt: 

In [ ]:
import os

file_path = "/content/output_wavs/2_1377G2A2_1376G2A2_T1_2D09T0417C000394_004992_000387.wav"
if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
    print(f"{file_path} is valid")
else:
    print(f"{file_path} is empty or does not exist")


/content/output_wavs/2_1377G2A2_1376G2A2_T1_2D09T0417C000394_004992_000387.wav is valid


# 2. 마지막 어절/음절 duration 추출 -whisper word timestemp
### 1) 마지막 어절

- whisper/notebooks/Multilingual_ASR.ipynb
  https://github.com/openai/whisper/blob/main/notebooks/Multilingual_ASR.ipynb

- whisperX

  https://github.com/m-bain/whisperX
  
  [참고] https://jellyfishdeveloper.tistory.com/entry/Whisper%EC%97%90-%EB%8B%A8%EC%96%B4%EB%B3%84-%ED%83%80%EC%9E%84%EC%8A%A4%ED%83%AC%ED%94%84%EA%B0%80-%ED%95%84%EC%9A%94%ED%95%A0-%EB%95%8C-whisperX-or-whisper-timestamped

### 2) 마지막 음절 -> 마지막 pitch 구간으로 대체

### 3) 어절과 억양, 그리고 태도
(1) "한국어 억양의 형태와 기능에 관한 연구", 이영근
- 청각상 구분이 어려운 태도들 사이에는 억양형태의 유사성이 있음
- 억양형태뿐 아니라 평균주파수, 길이, 주파수폭 등도 이들 태도들의 구분에 관여하고 있으므로 이 세 요소에 의해서 억양형태만으로는 구분이 어려운 태도들을 구분할 수 있음

(2) 그렇다면, pitch contour의 pitch slope뿐 아니라 평균주파수, 길이, 주파수폭, 이 세 요소를 특징 벡터로 넣어 주었을 때의 모델 성능 추이를 확인해볼 필요가 있음

-> 통계를 돌려서 각 요소를 특정 기준을 두고 구간을 나누거나 도색하여, 이미지로 넣어주면 어떨까? (인간 인지 수준에서의 class 별 차이를 색깔 지정을 통해 특징으로 넣어주는 느낌...)

### 2) 음운구와 음절 수, 그리고 억양
(2) "한국어 음운구 억양 유형의 변별적 특성과 변이 조건에 대한 연구: 형태소 경계의 영향을 중심으로", 오재혁
- 음절 수가 많을수록 낮게 시작하는 음운구에서는 음운구의 30% 지점의 음높이가 높아지고, 높게 시작하는 음운구에서는 음운구의 70% 지점의 음높이가 낮아진다. 한편 이처럼 음절 수 구성에 따라 영향을 받는 지점의 음높이 변이는 음절 수 구성보다 분절음 종류에 의한 영향이 더 앞선다.

# 2. column 추가: 전체 pitch 기울기, 시간 프레임별 억양
- 발화 내 전체 pitch 기울기 값 추출
- 각 시간 프레임에 대한 억양 주석(rising, falling)

In [ ]:


# 1. CSV 파일 로드
csv_file_path = '/content/filtered_train.csv'
df = pd.read_csv(csv_file_path)

# 새로운 폴더 경로 지정
output_folder = '/content/with_features'
os.makedirs(output_folder, exist_ok=True)

# 2. pitch 기울기 추출 및 억양 주석 달기
def extract_pitch_and_intonation(file_path):
    try:
        # librosa로 .wav 파일 로드
        y, sr = librosa.load(file_path)

        # Pitch 추출: librosa의 pitch 추정 함수 사용 (음성의 주파수 추정)
        pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)

        # 각 시간 프레임에서 가장 높은 pitch를 추출
        pitch = []
        for t in range(pitches.shape[1]):
            index = magnitudes[:, t].argmax()
            pitch.append(pitches[index, t])

        # pitch 기울기 계산
        pitch = np.array(pitch)
        pitch_slope = np.gradient(pitch)  # 기울기 추출

        # 억양 주석: pitch 변화량이 큰 부분을 강조
        peaks, _ = find_peaks(pitch_slope, height=0.1)  # 임계값은 조정 가능
        intonation_annotation = ['rising' if i in peaks else 'falling' for i in range(len(pitch_slope))]

        return pitch_slope, intonation_annotation
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return [], []

# 3. 각 파일에 대해 pitch 기울기와 억양 주석 추가
segment_pitch_slopes = []
segment_intonation_annotations = []

for index, row in df.iterrows():
    # row['path']의 ./train을 /content/train_data로 변환
    file_path = row['path'].replace('./train', '/content/train_data')
    print(f"Processing: {file_path}")  # 확인용 출력 (필요 시 제거)

    # pitch 기울기와 억양 주석 추출
    pitch_slope, intonation_annotation = extract_pitch_and_intonation(file_path)

    # 리스트에 추가
    segment_pitch_slopes.append(pitch_slope)
    segment_intonation_annotations.append(intonation_annotation)

# 4. 새로운 컬럼을 기존 DataFrame에 추가
df['segment_pitch_slope'] = segment_pitch_slopes
df['segment_intonation_annotations'] = segment_intonation_annotations

# 5. 새로운 .csv 파일로 저장
output_csv_path = os.path.join(output_folder, 'train_data_with_pitch_and_intonation.csv')
df.to_csv(output_csv_path, index=False)

print(f"새로운 데이터가 {output_csv_path}에 저장되었습니다.")
